In [41]:
# This notebook was used to develop the Django management command "compare_databases." That should be invoked directly rather than this notebook, which may no longer be up to date. 

In [1]:
import pandas as pd
import numpy as np
import config

In [2]:
import os
os.environ['CHILDES_EC2_PASS'] = config.CHILDES_EC2_PASS 
os.environ['ROOT_PASS'] = config.ROOT_PASS 

In [3]:
import compare_refactor

In [4]:
local_params = {
    "host": "127.0.0.1",
    "current":"childes_db_dev",
        "supported": ["childes_db_dev", "childes_db_dev_test2","2020.1"],
        "historical":[],
    "user": "root",
    "password": os.environ['ROOT_PASS']
}

In [21]:
old_db_name = "2020.1" # the previously obtained values
new_db_name = "childes_db_dev_test2" # the new values

old_db_con = compare_refactor.connect_to_childes(db_version = old_db_name, db_args = local_params)
new_db_con = compare_refactor.connect_to_childes(db_version = new_db_name, db_args = local_params)

In [17]:
# transcript_id from expected
# transcript_id from actual
# 1881,1878  # where do these two numbers come from?

old_db_transcript_id = 1881
new_db_transcript_id = 1878 

### Replicate the hash mismatch

In [7]:
import imp
imp.reload(compare_refactor)

<module 'compare_refactor' from '/home/stephan/notebooks/childes-db/compare_refactor.py'>

In [8]:
compare_refactor.compare_hashes(actual_transcript_id, expected_transcript_id, actual_con, expected_con, query = "SELECT * FROM token WHERE transcript_id = %s")

,actual,expected
0,2797135505886509492,17403552568618603677
1,6832313348406032358,14476633068354218331
2,14193210056305269719,18037000450445010434
3,445914329092122844,4807973286065834821
4,16235165885644159520,603863056013329225
5,10989139032648234986,15712189000685403203
6,13273142551762913159,3529977753542233966
7,8377235271152228385,13386587427833563640
8,9731390615221132823,11980439253443890486
9,3230031072501646871,397568033355613782


### Find the contents that are different

In [19]:
new_transcript_id = 1879
old_transcript_id = 1880
compare_refactor.compare_dfs(new_transcript_id, old_transcript_id, new_db_con, old_db_con, query = "SELECT * FROM token WHERE transcript_id = %s")


target_child_sex      
                self other
0             female  male
1             female  male
2             female  male
3             female  male
4             female  male
..               ...   ...
286           female  male
287           female  male
288           female  male
289           female  male
290           female  male

[291 rows x 2 columns]

### Run all discrepanices from the log

In [10]:
token_errors = pd.read_csv('/home/snair/childes-db/token_errors.log')
token_errors.head(5)

,main_id,refactor_id,local_fname,collection,empty_for_trnst
0,1881,1878,Frogs/English-MiamiMono/mleng2/31321413.xml,Frogs,neither
1,1880,1879,Frogs/English-MiamiMono/mleng2/31321414.xml,Frogs,neither
2,1876,1880,Frogs/English-MiamiMono/mleng2/31321319.xml,Frogs,neither
3,1879,1881,Frogs/English-MiamiMono/mleng2/31321347.xml,Frogs,neither
4,1882,1882,Frogs/English-MiamiMono/mleng2/31321445.xml,Frogs,neither


In [11]:
token_error_results = []
for transcript_row in token_errors.to_dict('records'):
    actual_transcript_id = transcript_row['refactor_id']
    expected_transcript_id = transcript_row['main_id']
    col_comparison = compare_refactor.compare_dfs(new_transcript_id, old_transcript_id, new_db_con, old_db_con, query = "SELECT * FROM token WHERE transcript_id = %s",
                                return_type = 'colnames')
    
    token_error_results.append(' '.join(col_comparison))    
token_error_results

['target_child_sex',
 'target_child_sex',
 'target_child_sex',
 'target_child_sex',
 'target_child_sex',
 'target_child_sex',
 'target_child_sex',
 'target_child_sex',
 'target_child_sex',
 'target_child_sex',
 'target_child_sex',
 'target_child_sex',
 'target_child_sex',
 'target_child_sex',
 'target_child_sex',
 'target_child_sex',
 'target_child_sex',
 'target_child_sex',
 'target_child_sex',
 'target_child_sex',
 'target_child_sex',
 'target_child_sex',
 'target_child_sex',
 'target_child_sex',
 'target_child_sex',
 'target_child_sex',
 'target_child_sex',
 'target_child_sex',
 'target_child_sex',
 'target_child_sex',
 'target_child_sex',
 'target_child_sex',
 'target_child_sex',
 'target_child_sex',
 'target_child_sex',
 'target_child_sex',
 'target_child_sex',
 'target_child_sex',
 'target_child_sex',
 'target_child_sex',
 'target_child_sex',
 'target_child_sex',
 'target_child_sex',
 'target_child_sex',
 'target_child_sex',
 'target_child_sex',
 'target_child_sex',
 'target_chil

In [12]:
token_errors_augmented = token_errors
token_errors['mismatched_columns'] = token_error_results
token_errors.to_csv('token_errors_augmented.log')


In [13]:
pd.Series(token_error_results).value_counts()

target_child_sex                                   1573
speaker_name target_child_name                      125
speaker_name target_child_name target_child_sex      66
speaker_name                                         45
dtype: int64

In [14]:
# [X] make a "compare hashes command"
# [X] Make a "compare dataframes" function that looks for differences programatically within the dataframe
# [X] See if gender is the common problem for other ones -- some of them at least
# [X] Move functions into the compare_refactor.py code
# [X] run this on the log output to see if we can identifiy what went wrong in each case

# Run the transcript comparison code across a corpus

In [36]:
import imp
imp.reload(compare_refactor)

<module 'compare_refactor' from '/home/stephan/notebooks/childes-db/compare_refactor.py'>

In [38]:
target_collection_name = "Frogs"
query = 'SELECT * FROM transcript WHERE collection_name = "'+target_collection_name+'"'
old_transcript_ids = pd.read_sql(query, old_db_con)
old_transcript_ids.head(5)



,id,corpus_name,language,date,filename,target_child_name,target_child_age,target_child_sex,collection_name,pid,collection_id,corpus_id,target_child_id
0,1,English-WolfHemp,eng,1989-06-14,Frogs/English-WolfHemp/06/31w6.xml,John,2374.1250,male,Frogs,11312/c-00020802-1,1,1,1.0
1,2,English-WolfHemp,eng,1989-09-06,Frogs/English-WolfHemp/06/34w6.xml,Rachel,2435.0000,female,Frogs,11312/c-00020808-1,1,1,2.0
2,3,English-WolfHemp,eng,1989-07-13,Frogs/English-WolfHemp/06/06m6.xml,Christina,2435.0000,female,Frogs,11312/c-00020784-1,1,1,5.0
3,4,English-WolfHemp,eng,None,Frogs/English-WolfHemp/06/02m6.xml,Margaret,2435.0000,female,Frogs,11312/c-00020775-1,1,1,3.0
4,5,English-WolfHemp,eng,1989-07-10,Frogs/English-WolfHemp/06/04w6.xml,Erin,2404.5625,female,Frogs,11312/c-00020781-1,1,1,4.0


In [39]:
new_transcript_ids = pd.read_sql(query, new_db_con)
new_transcript_ids.head(5)

,id,corpus_name,language,date,filename,target_child_name,target_child_age,target_child_sex,collection_name,pid,collection_id,corpus_id,target_child_id
0,1,English-WolfHemp,eng,1989-07-13,Frogs/English-WolfHemp/06/06m6.xml,Christina,2435.0000,female,Frogs,11312/c-00020784-1,1,1,1.0
1,2,English-WolfHemp,eng,1989-07-10,Frogs/English-WolfHemp/06/04w6.xml,Erin,2404.5625,female,Frogs,11312/c-00020781-1,1,1,36.0
2,3,English-WolfHemp,eng,None,Frogs/English-WolfHemp/06/02m6.xml,Margaret,2435.0000,female,Frogs,11312/c-00020775-1,1,1,3.0
3,4,English-WolfHemp,eng,1989-07-25,Frogs/English-WolfHemp/06/14w6.xml,Alexandra,2404.5625,female,Frogs,11312/c-00020790-1,1,1,4.0
4,5,English-WolfHemp,eng,1989-06-22,Frogs/English-WolfHemp/06/13w6.xml,Michael,2404.5625,male,Frogs,11312/c-00020787-1,1,1,7.0


In [40]:
merged = old_transcript_ids[['filename','id']].merge(new_transcript_ids[['filename','id']], how="outer", on=["filename"])
merged.columns = ['filename','old_transcript_id','new_transcript_id']
merged


,filename,old_transcript_id,new_transcript_id
0,Frogs/English-WolfHemp/06/31w6.xml,1,19
1,Frogs/English-WolfHemp/06/34w6.xml,2,17
2,Frogs/English-WolfHemp/06/06m6.xml,3,1
3,Frogs/English-WolfHemp/06/02m6.xml,4,3
4,Frogs/English-WolfHemp/06/04w6.xml,5,2
...,...,...,...
3254,Frogs/Chinese-Tardif/04/f3-21220108.xml,3255,3254
3255,Frogs/Chinese-Tardif/04/f3-21220111.xml,3256,3256
3256,Frogs/Chinese-Tardif/04/f3-21220114.xml,3257,3257
3257,Frogs/Chinese-Tardif/04/f3-21220118.xml,3258,3258


In [34]:
# check these pairs by matching; time it and see if it is better to parallelize it

for transcript_pair in merged.iloc[0:5].to_dict("records"):
    print(transcript_pair)

    compare.



{'filename': 'Frogs/English-WolfHemp/06/31w6.xml', 'old_transcript_id': 1, 'new_transcript_id': 19}
{'filename': 'Frogs/English-WolfHemp/06/34w6.xml', 'old_transcript_id': 2, 'new_transcript_id': 17}
{'filename': 'Frogs/English-WolfHemp/06/06m6.xml', 'old_transcript_id': 3, 'new_transcript_id': 1}
{'filename': 'Frogs/English-WolfHemp/06/02m6.xml', 'old_transcript_id': 4, 'new_transcript_id': 3}
{'filename': 'Frogs/English-WolfHemp/06/04w6.xml', 'old_transcript_id': 5, 'new_transcript_id': 2}
